In [83]:
from duckduckgo_search import ddg_images
from fastcore.all import *

def search_images(term, max_images=200): return L(ddg_images(term, max_results=max_images)).itemgot('image')

from fastdownload import download_url

from fastai.vision.all import *
import timm
import numpy as np
import requests
from bs4 import BeautifulSoup as bs4
import pandas as pd

In [5]:
page_t = requests.get('https://boardgamegeek.com/image/2669798')

In [6]:
soup_t = bs4(page_t.content, 'html.parser')

In [7]:
def extract_img(image_number): 
    page = requests.get('https://boardgamegeek.com/image/'+str(image_number))
    soup = bs4(page.content, 'html.parser')
    uncleaned_image_link=(str(soup.find('link',href=re.compile(r'geekdo-images'))))
    patt= re.compile(r'https.*'+str(image_number)+r'[^"]*')
    image_link=patt.findall(uncleaned_image_link) #add an assertion that the list is nonempty
    
    uncleaned_game_name=(str(soup.find('link',rel='canonical')))
    patt2= re.compile(str(image_number)+r'/[^"]*')
    game_name=patt2.findall(uncleaned_game_name) #add an assertion that the list is nonempty
    if len(image_link)==0 or len(game_name)==0:
        return 'invalid', uncleaned_image_link +'\n' + uncleaned_game_name +'\n' +str(soup)
    return game_name[0][8:], image_link[0]  #,page, soup, uncleaned_image_link, patt,

In [8]:
page_t = requests.get('https://boardgamegeek.com/image/'+str(2669812))
soup_t = bs4(page_t.content, 'html.parser')
uncleaned_image_link_t=(str(soup_t.find('link',href=re.compile(r'geekdo-images'))))
patt_t= re.compile(r'https.*'+str(2669812)+r'[^"]*')
image_link_t=patt_t.findall(uncleaned_image_link_t)

In [9]:
page_t.status_code==404

True

In [10]:
extract_img(2669798)

('gloomhaven',
 'https://cf.geekdo-images.com/HNMHfQ49oxs8ig-IEmNQWQ__imagepage/img/nNo4l2NjXToU3DCoBnMIbDExJu0=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2669798.png')

In [11]:
extract_img(2669900)

('jmaydet',
 'https://cf.geekdo-images.com/0WHEngboBp96fNQmqVNjiQ__imagepage/img/95blS1S21dYclnS_bMi7ws_pRq8=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2669900.jpg')

In [75]:
def get_links_simple(start,stop):
    linklist=[]
    errorlist=[]
    for image_number in range(start,stop): # can vectorize this using numpy - output is then first a single vector that we split into links and errors afterwards
        if image_number%100==0 and image_number!=0:
                print(image_number)
        page = requests.get('https://boardgamegeek.com/image/'+str(image_number))
        if page.status_code==404:
            errorlist.append([image_number,404,None])#None for vectorization
        else:
            soup = bs4(page.content, 'html.parser')
            uncleaned_image_link=(str(soup.find('link',href=re.compile(r'geekdo-images')))) #isolates the full link element to the image
            patt= re.compile(r'https.*'+str(image_number)+r'[^"]*')  
            image_link=patt.findall(uncleaned_image_link) #removes everything except the link iteself

            uncleaned_game_name=(str(soup.find('link',rel='canonical'))) 
            patt2= re.compile(str(image_number)+r'/[^"]*')
            game_name=patt2.findall(uncleaned_game_name)
            
            if image_link==[] or game_name==[]:
                errorlist.append([image_number, page.status_code,None]) #None for vectorization
            else: 
                linklist.append([image_number,game_name[0][8:],image_link[0]])
            
    return [linklist, errorlist]

In [40]:
l1 = get_links(2669798,2669806)

2669800


In [41]:
l1

([[2669798,
   'gloomhaven',
   'https://cf.geekdo-images.com/HNMHfQ49oxs8ig-IEmNQWQ__imagepage/img/nNo4l2NjXToU3DCoBnMIbDExJu0=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2669798.png'],
  [2669799,
   'paying-peiper',
   'https://cf.geekdo-images.com/HOsk9e2uJ9WxZoSMT0RMxQ__imagepage/img/bl0d18X72meL8hTCf4isAoA1uvE=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2669799.jpg'],
  [2669800,
   'paying-peiper',
   'https://cf.geekdo-images.com/OLOEUjwxOoePxzZYI6eWFA__imagepage/img/wVft8ypQmpBmD7gMWMuvgBIQ_YI=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2669800.jpg'],
  [2669801,
   'rayz2112',
   'https://cf.geekdo-images.com/rlKIMylDNnmAqtp-n_DvyA__imagepage/img/88bhzKCFSKWNtvgJSPGOjFqqLfc=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2669801.jpg'],
  [2669802,
   'mystery-abbey',
   'https://cf.geekdo-images.com/YaUXlvHosnsl1kul2Y9ctQ__imagepage/img/HeErbWb2ooRBTubTsqXp0lB5JeI=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2669802.jpg'],
  [2669803,
   'bu

In [42]:
l2 = get_links(2669806,2669816)

In [43]:
l2

([[2669806,
   'jrtracy',
   'https://cf.geekdo-images.com/YCweY_9JFsQRKXdk29ngRQ__imagepage/img/5DoGDinpQ-L_1ig2EzTsjrhlNB8=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2669806.jpg'],
  [2669807,
   'advanced-bernard-jean-deuxieme-edition',
   'https://cf.geekdo-images.com/G5jo2pjgd0nEp87P8XSabw__imagepage/img/HdarJ6bTxwGgQXeWFHF7qmR3B7Y=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2669807.jpg'],
  [2669808,
   '2tomatoes-games',
   'https://cf.geekdo-images.com/CCuAdnh3YNAppn1It5kVIQ__imagepage/img/2BJeY96lIwqiGfw1MF7o4MsFpH4=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2669808.png'],
  [2669809,
   'yellow-tome-beyond-mists',
   'https://cf.geekdo-images.com/EkgxFg18utzTEik5QOrNkg__imagepage/img/raZYRidljUCKiqcpWTS9mWgQbYg=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2669809.png'],
  [2669810,
   'blur-hand-quicker-die',
   'https://cf.geekdo-images.com/DNGILi56eeCzJcflHKjKag__imagepage/img/iT_H5bt2raRHVhRN1KOAgaWLYNw=/fit-in/900x600/filters:no_upscale():

In [44]:
l1

([[2669798,
   'gloomhaven',
   'https://cf.geekdo-images.com/HNMHfQ49oxs8ig-IEmNQWQ__imagepage/img/nNo4l2NjXToU3DCoBnMIbDExJu0=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2669798.png'],
  [2669799,
   'paying-peiper',
   'https://cf.geekdo-images.com/HOsk9e2uJ9WxZoSMT0RMxQ__imagepage/img/bl0d18X72meL8hTCf4isAoA1uvE=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2669799.jpg'],
  [2669800,
   'paying-peiper',
   'https://cf.geekdo-images.com/OLOEUjwxOoePxzZYI6eWFA__imagepage/img/wVft8ypQmpBmD7gMWMuvgBIQ_YI=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2669800.jpg'],
  [2669801,
   'rayz2112',
   'https://cf.geekdo-images.com/rlKIMylDNnmAqtp-n_DvyA__imagepage/img/88bhzKCFSKWNtvgJSPGOjFqqLfc=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2669801.jpg'],
  [2669802,
   'mystery-abbey',
   'https://cf.geekdo-images.com/YaUXlvHosnsl1kul2Y9ctQ__imagepage/img/HeErbWb2ooRBTubTsqXp0lB5JeI=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2669802.jpg'],
  [2669803,
   'bu

In [52]:
def get_links(image_number): #vectorization?
    page = requests.get('https://boardgamegeek.com/image/'+str(image_number))
    if page.status_code==404:
        game_name='404'
        image_link=None
    else:
        soup = bs4(page.content, 'html.parser')
        uncleaned_image_link=(str(soup.find('link',href=re.compile(r'geekdo-images')))) #isolates the full link element to the image
        patt= re.compile(r'https.*'+str(image_number)+r'[^"]*')  
        image_link=patt.findall(uncleaned_image_link) #removes everything except the link iteself

        uncleaned_game_name=(str(soup.find('link',rel='canonical'))) 
        patt2= re.compile(str(image_number)+r'/[^"]*')
        game_name=patt2.findall(uncleaned_game_name)

        if image_link==[] or game_name==[]:
            game_name=str(page.status_code)
            image_link= None
        else: 
            game_name=game_name[0][8:]
            image_link=image_link[0]

    return np.asarray([image_number,game_name,image_link])

In [56]:
l1=get_links_simple(2669798,2669808)

2669800


In [57]:
l2=get_links_simple(2669798,2669898)

2669800


This took ~30 seconds

In [58]:
60*2*100

12000

So I could expect to get 12000 links in an hour

In [80]:
gns=np.asarray(l2[0])[:,1]

In [94]:
pd.DataFrame(gns).value_counts()

esbegan                                               8
pickboy87                                             8
imperios-milenarios                                   4
legacy-testament-duke-de-crecy-five-families          4
manuel-du-gendarme-et-du-voleur                       3
castles-mad-king-ludwig                               3
camel                                                 3
manuel-du-meneur-de-jeu                               3
livret-denquete                                       3
lefty-brune                                           3
blur-hand-quicker-die                                 2
martinus                                              2
paying-peiper                                         2
basic-pulp                                            2
yellow-tome-beyond-mists                              2
concordia                                             2
curses-vampire                                        2
jrtracy                                         

We need an effective way to isolate games from users

# 